In [13]:
import amazon_reviews
import openai
import json
import os

openai.api_key  = ''

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [14]:
amazon, amazon_value = amazon_reviews.search()

b = amazon_value

json_file = amazon

with open(json_file, "r") as file:
    data = json.load(file)

reviews = []

for review_list in data:
    for review in review_list:
        reviews.append(review['body'])


In [15]:
prompt = f"""

Analyze the following collection of reviews and employ topic modeling techniques to categorize the feedback into specific features of the product.
Divide each feature in positive characteristics and in negative characteristics.
Response format: 
                Feature:       
                    -Name: x
                    -Product name: q (your own words)
                    -Positive characteristics: y (your own words)
                    -Negative characteristics: z (your own words)
                    -If there are no positive or negative characteristics, write "Not applicable".

Provide it in JSON format to save in JSON file.

Review text: '''{reviews}'''
"""
response = get_completion(prompt)

with open('response.json', 'w', encoding='utf-8') as f:
    f.write(response)

print(response, "\n")

{
    "Features": [
        {
            "Name": "Button size",
            "Product name": "Microwave",
            "Positive characteristics": "Not applicable",
            "Negative characteristics": "Buttons quite small"
        },
        {
            "Name": "Cooking evenly",
            "Product name": "Microwave",
            "Positive characteristics": "Not applicable",
            "Negative characteristics": "Not convinced that any small microwave cooks as evenly as a full size one"
        },
        {
            "Name": "Power",
            "Product name": "Microwave",
            "Positive characteristics": "Good, powerful",
            "Negative characteristics": "Not applicable"
        },
        {
            "Name": "Ease of use",
            "Product name": "Microwave",
            "Positive characteristics": "Stylish and easy to use, one touch and it's done, easy to use with preset buttons, great purchase, easy to operate and works well",
            "Negative ch

In [16]:
import json
import dominate
from dominate.tags import *

with open('response.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

doc = dominate.document(title='Features')

with doc.head:
    style("""
        body {
            font-family: Arial, sans-serif;
        }
        .feature {
            border: 1px solid #ddd;
            margin-bottom: 10px;
            padding: 10px;
            border-radius: 5px;
        }
        .feature-name {
            font-size: 20px;
            color: #444;
        }
        .positive {
            color: green;
        }
        .negative {
            color: red;
        }
    """)

with doc:
    product_name = data['Features'][0]['Product name']
    h1(f' {product_name} - Product ID: {amazon_value}', style="text-align:center;")
    for feature in data['Features']:
        with div(cls='feature'):
            h2(feature['Name'], cls='feature-name')
            with div(cls='positive'):
                h3('Positive characteristics:')
                ul(li(pc) for pc in [feature['Positive characteristics']])
            with div(cls='negative'):
                h3('Negative characteristics:')
                ul(li(nc) for nc in [feature['Negative characteristics']])


with open('output.html', 'w', encoding='utf-8') as f:
    f.write(str(doc))